**Aditya Dwi Aryanto | 103132400027**

**Fauzi Romadhoni    | 103132400025**

**Link Dataset : https://drive.google.com/drive/folders/19JIdAJ9V0Bsc_SsOHyApMobJZbg4-Sq-?usp=sharing**

**Membaca Data dari File**

In [23]:
import csv

def load_and_clean_data(file_path):
    dataset = []
    seen_ids = set()

    with open(file_path, mode='r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # Mengabaikan baris jika ID sudah ada
            c_id = row['Customer ID']
            if c_id in seen_ids or not c_id:
                continue

            # menangani missing values
            if not row['Satisfaction Score'] or not row['Purchase Amount']:
                continue

            try:
                item = {
                    'id': c_id,
                    'score': float(row['Satisfaction Score']),
                    'amount': float(row['Purchase Amount'])
                }
                dataset.append(item)
                seen_ids.add(c_id)
            except ValueError:

                continue
    return dataset

# Inisialisasi data
raw_data = load_and_clean_data('customer_satisfaction_data.csv')
print(f"Data berhasil dimuat: {len(raw_data)} baris.")

Data berhasil dimuat: 859 baris.


**Fuzzification**

In [24]:
def get_membership_satisfaction(val):
    # Linguistik: Buruk, Biasa, Puas
    memberships = {'buruk': 0, 'biasa': 0, 'puas': 0}

    # Fungsi keanggotaan untuk 'buruk' (skala 1-12)
    if val <= 4:
        memberships['buruk'] = 1
    elif 4 < val < 6:
        memberships['buruk'] = (6 - val) / (6 - 4)

    # Fungsi keanggotaan untuk 'biasa'
    if 4 < val <= 6:
        memberships['biasa'] = (val - 4) / (6 - 4)
    elif 6 < val <= 8:
        memberships['biasa'] = 1
    elif 8 < val < 10:
        memberships['biasa'] = (10 - val) / (10 - 8)

    # Fungsi keanggotaan untuk 'puas'
    if 8 < val <= 10:
        memberships['puas'] = (val - 8) / (10 - 8)
    elif val > 10:
        memberships['puas'] = 1

    return memberships

def get_membership_purchase(val):
    # Linguistik: Hemat, Normal, Loyal
    memberships = {'hemat': 0, 'normal': 0, 'loyal': 0}

    # Skala ditentukan berdasarkan persebaran data transaksi
    if val <= 250:
        memberships['hemat'] = 1
    elif 250 < val < 450:
        memberships['hemat'] = (450 - val) / (450 - 250)

    if 250 < val <= 450:
        memberships['normal'] = (val - 250) / (450 - 250)
    elif 450 < val <= 700:
        memberships['normal'] = 1
    elif 700 < val < 900:
        memberships['normal'] = (900 - val) / (900 - 700)

    if 700 < val <= 900:
        memberships['loyal'] = (val - 700) / (900 - 700)
    elif val > 900:
        memberships['loyal'] = 1

    return memberships

**Inferensi**

In [25]:
def inference_logic(satisfaction, purchase):
    # Target: Menentukan derajat Prioritas Rendah dan Prioritas Tinggi
    priority_low = []
    priority_high = []

    # Aturan 1: Jika Puas DAN Loyal maka Prioritas Tinggi
    priority_high.append(min(satisfaction['puas'], purchase['loyal']))

    # Aturan 2: Jika Puas DAN Normal maka Prioritas Tinggi
    priority_high.append(min(satisfaction['puas'], purchase['normal']))

    # Aturan 3: Jika Buruk DAN Hemat maka Prioritas Rendah
    priority_low.append(min(satisfaction['buruk'], purchase['hemat']))

    # Aturan 4: Jika Biasa DAN Hemat maka Prioritas Rendah
    priority_low.append(min(satisfaction['biasa'], purchase['hemat']))

    # Agregasi menggunakan nilai maksimum
    return max(priority_low), max(priority_high)

**Inferensi**

In [26]:
def inference_logic(satisfaction, purchase):
    # Target: Menentukan derajat Prioritas Rendah dan Prioritas Tinggi
    priority_low = []
    priority_high = []

    # Aturan 1: Jika Puas DAN Loyal maka Prioritas Tinggi
    priority_high.append(min(satisfaction['puas'], purchase['loyal']))

    # Aturan 2: Jika Puas DAN Normal maka Prioritas Tinggi
    priority_high.append(min(satisfaction['puas'], purchase['normal']))

    # Aturan 3: Jika Buruk DAN Hemat maka Prioritas Rendah
    priority_low.append(min(satisfaction['buruk'], purchase['hemat']))

    # Aturan 4: Jika Biasa DAN Hemat maka Prioritas Rendah
    priority_low.append(min(satisfaction['biasa'], purchase['hemat']))

    # Agregasi menggunakan nilai maksimum
    return max(priority_low), max(priority_high)

**Defuzzification**

In [27]:
def calculate_defuzzification(low_val, high_val):
    # Nilai konstanta output: Rendah = 40, Tinggi = 95
    weight_low = 40
    weight_high = 95

    numerator = (low_val * weight_low) + (high_val * weight_high)
    denominator = low_val + high_val

    if denominator == 0:
        return 0

    return numerator / denominator

**menyimpan Output ke File**

In [28]:
def export_results(data, output_file):
    processed_results = []

    for item in data:
        f_satisfaction = get_membership_satisfaction(item['score'])
        f_purchase = get_membership_purchase(item['amount'])

        low_deg, high_deg = inference_logic(f_satisfaction, f_purchase)
        final_score = calculate_defuzzification(low_deg, high_deg)

        processed_results.append({
            'id': item['id'],
            'satisfaction': item['score'],
            'purchase': item['amount'],
            'final_score': final_score
        })

    # Mengurutkan berdasarkan skor akhir tertinggi
    processed_results.sort(key=lambda x: x['final_score'], reverse=True)

    # Menampilkan 5 terbaik di konsol
    print("Daftar 5 Pelanggan Terbaik:")
    for i in range(min(5, len(processed_results))):
        p = processed_results[i]
        print(f"ID: {p['id']} | Skor: {p['final_score']:.2f}")

    # Menulis ke file CSV
    with open(output_file, mode='w', newline='') as f:
        fields = ['id', 'satisfaction', 'purchase', 'final_score']
        writer = csv.DictWriter(f, fieldnames=fields)
        writer.writeheader()
        writer.writerows(processed_results)

# Eksekusi akhir
export_results(raw_data, 'hasil_fuzzy_pelanggan.csv')

Daftar 5 Pelanggan Terbaik:
ID: CUST0926 | Skor: 95.00
ID: CUST0429 | Skor: 95.00
ID: CUST0694 | Skor: 95.00
ID: CUST0682 | Skor: 95.00
ID: CUST0379 | Skor: 95.00
